<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/SOES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_md

     |████████████████████████████████| 78.4MB 1.3MB/s 
  Created wheel for es-core-news-md: filename=es_core_news_md-2.2.5-cp36-none-any.whl size=79649483 sha256=3d8e8e9116972f3ef9b454fe9604edd9dd82be4a53063e200dbb8591c0bf91b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0m8kd4v/wheels/b7/bb/a3/29ab5cf80c2c0a8fa0f2af8402fdace3f159e8265f0fdcbcdb
Successfully built es-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from time import time
start_nb = time()

start = time()
import pandas as pd
from nltk import SnowballStemmer
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

import es_core_news_md
nlp = es_core_news_md.load()

es_stops = set(stopwords.words('spanish'))
Snowball_stemmer = SnowballStemmer('spanish')

print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 21.85 segundos


# PRE-PROCESSED

### DATASET

In [ ]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
dataset_size = len(dataset)


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 0.68 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


### LEMATIZACIÓN

In [ ]:
def preprocess_lemma(doc):
    separator = ' '
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = separator.join(doc)
    doc = nlp(doc)
    doc = [w.lemma_ for w in doc]
    return doc

In [ ]:
start = time()
dataset['Lemma_Title'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Title']), axis=1)
dataset['Lemma_Body'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Body']), axis=1)
print('Dataset generado. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Dataset generado. 34.01 segundos


,Lemma_Title,Lemma_Body
0,"[debugear, código, django, utilizar, docker]","[contar, realizar, proyectar, django, utilizar..."
1,"[mostrar, error, usar, supervisor, digital, oc...","[configurar, ruta, sudar, nano, program, factu..."
2,"[ejecutar, navegador, pantalla, completo, inic...","[realizar, proyectar, domotico, siguiente, com..."
3,"[paginar, web, blanco]","[bueno, dia, paginar, web, salir, blanco, algu..."
4,"[obtener, error, skipped, frames, the, applica...","[trabajar, android, studio, intentar, hacer, c..."
...,...,...
1079,"[conexion, remoto, sql, server, linux, mint]","[instalar, sql, server, linux, mint, basar, da..."
1080,"[crear, basar, dato, centos, funcionar]","[utilizar, centos, seguir, instrucción, sitiar..."
1081,"[hacer, respaldar, basar, dato, terminal]","[basar, dato, milisegundo, sql, server, expres..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollar, utilizar, cámara, es..."


### STEMMER

In [ ]:
def preprocess_stem(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = [Snowball_stemmer.stem(w) for w in doc]  # Extraer raices o stems (Stemming)
    return doc

In [ ]:
start = time()
dataset['Stem_Title'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Title']), axis=1)
dataset['Stem_Body'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[5,6]]

Preprocesado completo. 3.78 segundos


,Stem_Title,Stem_Body
0,"[debug, codig, djang, utiliz, dock]","[cuent, realiz, proyect, djang, utiliz, pychar..."
1,"[muestr, error, usand, supervisor, digital, oc...","[configur, rut, sud, nan, program, facturaempr..."
2,"[ejecut, naveg, pantall, complet, inici, linux]","[realiz, proyect, domot, siguient, component, ..."
3,"[pagin, web, blanc]","[buen, dia, pagin, web, sal, blanc, algui, pod..."
4,"[obteng, error, skipp, fram, the, application,...","[trabaj, andro, studi, intent, hac, conexion, ..."
...,...,...
1079,"[conexion, remot, sql, serv, linux, mint]","[instal, sql, serv, linux, mint, bas, dat, cre..."
1080,"[cre, bas, dat, cent, funcion]","[utiliz, cent, segu, instruccion, siti, micros..."
1081,"[hac, respald, bas, dat, terminal]","[bas, dat, ms, sql, serv, express, ubuntu, adm..."
1082,"[aplic, windows, linux, win]","[aplic, desarroll, utiliz, cam, escan, lector,..."


### PRE-PROCESADO (SOLO STOPWORD & SIGNOS DE PUNTUACIÓN)

In [ ]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[7,8]]

Preprocesado completo. 1.69 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[muestra, error, usando, supervisor, digital, ...","[configurar, ruta, sudo, nano, program, factur..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, skipped, frames, the, applica...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint]","[instalado, sql, server, linux, mint, base, da..."
1080,"[crear, base, datos, centos, funciona]","[utilizando, centos, seguí, instrucciones, sit..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, express, ubuntu..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, utiliza, cámara, es..."


# WORD MOVER DISTANCE (WMD)

In [ ]:
from sklearn.model_selection import train_test_split 

### WMD - LEMMA

In [ ]:
train_l, test_l = train_test_split(dataset['Lemma_Title'], test_size = 0.20, shuffle = False)

print("(Lemma) Ejemplos usados para entrenar: ", len(train_l))
print("(Lemma) Ejemplos usados para test: ", len(test_l))

(Lemma) Ejemplos usados para entrenar:  867
(Lemma) Ejemplos usados para test:  217


In [ ]:
test_l

867                 [backup, postgresql, error, password]
868     [ejecutar, sentenciar, postgresql, líneo, coma...
869                          [raspberry, pi, kali, linux]
870     [existir, alguno, alternativo, raspberry, pi, ...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remoto, sql, server, linux, mint]
1080              [crear, basar, dato, centos, funcionar]
1081            [hacer, respaldar, basar, dato, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicitar, servidor, usuario, ftp,...
Name: Lemma_Title, Length: 217, dtype: object

In [ ]:
lemma_title = dataset['Lemma_Title'].to_list()

title = dataset['Title'].to_list()

In [ ]:
from gensim.models import Word2Vec

model_l = Word2Vec(lemma_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_l = WmdSimilarity(test_l, model_l, num_best)
print(instancia_l)

WmdSimilarity<217 docs, 100 features>


In [ ]:
start = time()

msj_l = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_l = preprocess_lemma(msj_l)

sim_l = instancia_l[bus_l]
print(len(sim_l))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.11 seconds to run.


In [ ]:
print ('Busqueda: ', msj_l)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_l[i][0]],'---> sim = %.4f' % sim_l[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5331
¿Como agregar varios puertos al mismo contenedor de Docker? ---> sim = 0.5294
Como ejecutar mas de un archivo que esta en un parámetro de jenkins ---> sim = 0.5230
grep combinado con echo ---> sim = 0.5230
bash script mostrar información de procesos ---> sim = 0.5229
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.5227
Problema con git en local ---> sim = 0.5222
¿Qué puerto TCP usa el programa ping? ---> sim = 0.5206
Pantalla completa Kali Linux VirtualBox? ---> sim = 0.5177
Instalar entorno grafico en VPS Linux? ---> sim = 0.5166


### WMD - STEMMER

In [ ]:
train_s, test_s = train_test_split(dataset['Stem_Title'], test_size = 0.20, shuffle = False)

print("(Stemm) Ejemplos usados para entrenar: ", len(train_s))
print("(Stemm) Ejemplos usados para test: ", len(test_s))

(Stemm) Ejemplos usados para entrenar:  867
(Stemm) Ejemplos usados para test:  217


In [ ]:
stemm_title = dataset['Stem_Title'].to_list()

title = dataset['Title'].to_list()

In [ ]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_s = Word2Vec(stemm_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_s = WmdSimilarity(test_s, model_s, num_best)
print(instancia_s)

WmdSimilarity<217 docs, 100 features>


In [ ]:
start = time()

msj_s = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_s = preprocess_stem(msj_s)

sim_s = instancia_s[bus_s] 
print(len(sim_s))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.09 seconds to run.


In [ ]:
print ('Busqueda: ', msj_s)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_s[i][0]],'---> sim = %.4f' % sim_s[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.6383
Cambiar permisos de carpetas compartidas con docker-compose ---> sim = 0.6328
¿como puedo solucionar esto kernel panic not syncing vfs unable en parrot? ---> sim = 0.6328
Problema server vsftpd linux ---> sim = 0.6244
Pantalla completa Kali Linux VirtualBox? ---> sim = 0.6233
moodle apache centos 7 ---> sim = 0.6228
¿Como hago para reinstalar libbz2.so.1.0? ---> sim = 0.5738
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5670
Obtengo el error I/Choreographer: Skipped 110 frames! The application may be doing too much work on its main thread ---> sim = 0.5579
Las siguientes advertencias me da mysqltuner ---> sim = 0.5542


### WMD - TEXT

In [ ]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
train_p, test_p = train_test_split(dataset['Processed_Title'], test_size = 0.20, shuffle = False)

print("Ejemplos usados para entrenar: ", len(train_p))
print("Ejemplos usados para test: ", len(test_p))

Ejemplos usados para entrenar:  867
Ejemplos usados para test:  217


In [ ]:
test_p

867                 [backup, postgresql, error, password]
868     [ejecutar, sentencias, postgresql, línea, coma...
869                          [raspberry, pi, kali, linux]
870     [existe, alguna, alternativa, raspberry, pi, c...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remota, sql, server, linux, mint]
1080               [crear, base, datos, centos, funciona]
1081             [hacer, respaldo, base, datos, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicita, servidor, usuario, ftp, ...
Name: Processed_Title, Length: 217, dtype: object

In [ ]:
processed_title = dataset['Processed_Title'].to_list()

title = dataset['Title'].to_list()

In [ ]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_p = Word2Vec(processed_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_p = WmdSimilarity(train_p, model_p, num_best=10)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>


In [ ]:
start = time()

msj_p = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_p = preprocess(msj_p)

sim_p = instancia_p[bus_p]
print(len(sim_p))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.30 seconds to run.


In [ ]:
print ('Busqueda: ', msj_p)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian ---> sim = 1.0000
Tengo un error constante en Debian cuando quiero actualizar mi sistema ---> sim = 0.6509
Error: spawn EACCES -- Derivados debian -- ionic ---> sim = 0.6067
I/O Error: Sistema de archivos de solo lectura en SSD (XUBUNTU) ---> sim = 0.6061
Error al instalar php-db en Debian 10 Buster ---> sim = 0.5958
Bash Script Linux Error al pasarle argumentos ---> sim = 0.5173
¿Cómo montar un disco duro NTFS en Linux (Debian)? ---> sim = 0.5164
¿Como correjir el Error al instala el editor sublime text desde repositorios ppa en las distribuciones Linux Mint o Lubuntu? ---> sim = 0.5145
Como Abrir el puerto 53 udp en un servidor debian 8 ---> sim = 0.5138
Error inicio de sesión, Linux Deepin 15.5 ---> sim = 0.5129


In [ ]:
dataset.iloc[:,[3,5,7]]

,Lemma_Title,Stem_Title,Processed_Title
0,"[debugear, código, django, utilizar, docker]","[debug, codig, djang, utiliz, dock]","[debugear, código, django, utilizando, docker]"
1,"[mostrar, error, usar, supervisor, digital, oc...","[muestr, error, usand, supervisor, digital, oc...","[muestra, error, usando, supervisor, digital, ..."
2,"[ejecutar, navegador, pantalla, completo, inic...","[ejecut, naveg, pantall, complet, inici, linux]","[ejecutar, navegador, pantalla, completa, inic..."
3,"[paginar, web, blanco]","[pagin, web, blanc]","[pagina, web, blanco]"
4,"[obtener, error, skipped, frames, the, applica...","[obteng, error, skipp, fram, the, application,...","[obtengo, error, skipped, frames, the, applica..."
...,...,...,...
1079,"[conexion, remoto, sql, server, linux, mint]","[conexion, remot, sql, serv, linux, mint]","[conexion, remota, sql, server, linux, mint]"
1080,"[crear, basar, dato, centos, funcionar]","[cre, bas, dat, cent, funcion]","[crear, base, datos, centos, funciona]"
1081,"[hacer, respaldar, basar, dato, terminal]","[hac, respald, bas, dat, terminal]","[hacer, respaldo, base, datos, terminal]"
1082,"[aplicación, windows, linux, wine]","[aplic, windows, linux, win]","[aplicación, windows, linux, wine]"


# VECTOR SPACE MODEL (VSM)

In [ ]:
from collections import Counter
import numpy as np
import math

p_title = dataset['Processed_Title'].to_list()
p_body = dataset['Processed_Body'].to_list()

N = len(dataset) # N = Cantidad de documentos
alpha = 0.3
DF = {}

for i in range(N):
  for w in p_title[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}
  for w in p_body[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}

for i in DF:
        DF[i] = len(DF[i])

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

In [ ]:
def doc_freq(DF, word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
    start = time()

    doc = 0
    tf_idf_title = {}

    for i in range(N):
        tokens = p_body[i]
        counter = Counter(tokens + p_title[i])
        words_count = len(tokens + p_title[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = np.log((N + 1) / (df + 1))
            tf_idf_title[doc, token] = tf * idf
        doc += 1

    doc = 0
    tf_idf_body = {}

    for i in range(N):
        tokens = p_title[i]
        counter = Counter(tokens + p_body[i])
        words_count = len(tokens + p_body[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            # numerator is added 1 to avoid negative values
            idf = np.log((N + 1) / (df + 1))  
            tf_idf_body[doc, token] = tf * idf
        doc += 1

    for i in tf_idf_title:
        tf_idf_title[i] *= alpha

    for i in tf_idf_body:
        tf_idf_title[i] = tf_idf_body[i]

    # D = Matriz TF-IDF de terminos de documentos
    D = np.zeros((N, total_vocab_size)) 
    for i in tf_idf_title:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf_title[i]
        except:
            pass

    print('Matriz TF-IDF de terminos de documentos generada. %.2f segundos' % (time() - start))
    print(D[0])
    print(D[1])

Matriz TF-IDF de terminos de documentos generada. 2.55 segundos
[0.16144072 0.05125391 0.12287463 ... 0.         0.         0.        ]
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
def print_doc(dataset, id):
    with pd.option_context('display.max_colwidth', 500):
        print(dataset.iloc[[id], [0,1]])

In [ ]:
start = time()

def gen_vector(tokens):
        Q = np.zeros((len(total_vocab)))
        counter = Counter(tokens)
        words_count = len(tokens)
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = math.log((N + 1) / (df + 1))
            try:
                ind = total_vocab.index(token)
                Q[ind] = tf * idf
            except:
                pass
        return Q

def cosine_similarity(k, query):
        tokens = preprocess(query)
        print("\nQuery:", query)
        d_cosines = []
        query_vector = gen_vector(tokens)
        for d in D:
            d_cosines.append(cosine_sim(query_vector, d))
        index_sort = np.array(d_cosines).argsort()[-k:][::-1]
        value_array = np.array(d_cosines)
        value_sort = np.sort(value_array)[-k:][::-1]
        res = []
        for i in range(k):
            res.append((index_sort[i], value_sort[i]))
        return res

results = cosine_similarity(5, "Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian")

print("\n")
for i in range(len(results)):
  print_doc(dataset, results[i][0])
  print("sim: ",results[i]) 
  print("-------------------------------------------------------")

print('Tiempo de ejecución: %.2f segundos' % (time() - start))


Query: Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian


         Id                                                                                 Title
232  306013  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
sim:  (232, 0.423063333240842)
-------------------------------------------------------
         Id                                                                            Title
868  282752  ¿Como ejecutar sentencias postgresql desde la línea de comandos en CentOs 6.10?
sim:  (868, 0.2507828665460338)
-------------------------------------------------------
         Id                                                                               Title
1008  50954  ¿Conectar PHP linux a una base de datos alojada en el s.o windows (DB POSTGRESQL)?
sim:  (1008, 0.24049054527499789)
-------------------------------------------------------
       Id                                     Title
741  2960  Aut

# TEST

In [75]:
start = time()
data = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
data_size = len(data)


print('Dataset generado. %.2f segundos' % (time() - start))
data.head()

Dataset generado. 0.04 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [393]:
def preprocess_test(doc):
    label = re.compile(r'\d+')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    # doc = word_tokenize(doc)  # Dividir en palabras.
    # doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    # doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [394]:
start = time()
data = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')


print('Dataset generado. %.2f segundos' % (time() - start))
data.head()

Dataset generado. 0.04 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


In [395]:
test = dataset['Title'].to_list()
data['Processed_Title'] = data.apply(
        lambda data: preprocess_test(data['Title']), axis=1)

data.iloc[:,[1,3]]

,Title,Processed_Title
0,Debugear código Django utilizando Docker,debugear código django utilizando docker
1,En el file.conf muestra Error usando Superviso...,en el file.conf muestra error usando superviso...
2,Ejecutar navegador en pantalla completa al ini...,ejecutar navegador en pantalla completa al ini...
3,Pagina web en blanco,pagina web en blanco
4,Obtengo el error I/Choreographer: Skipped 110 ...,obtengo el error i/choreographer: skipped fra...
...,...,...
1079,Conexion remota de SQL Server linux mint 19,conexion remota de sql server linux mint
1080,Crear base de datos en mssql-server en centos ...,crear base de datos en mssql-server en centos ...
1081,Como hacer un respaldo de la base de datos des...,como hacer un respaldo de la base de datos des...
1082,Aplicación de Windows en Linux con WINE,aplicación de windows en linux con wine


In [397]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import string

punct = string.punctuation
com = "''¿¡"
punct = punct + com
print(punct)

new_stops = set(stopwords.words('spanish') + list(punct))
print(new_stops)

# Create the Document Term Matrix
vectorizer = CountVectorizer(stop_words=new_stops, tokenizer=word_tokenize, preprocessor=preprocess_test)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~''¿¡
{'tenías', 'sería', 'hubieses', 'habríamos', 'sentid', 'esté', 'tuvieron', '}', 'tuyas', 'estaremos', 'estaríais', 'es', 'hube', 'tendríamos', 'ante', 'él', 'lo', 'habíamos', 'esto', '¡', 'todo', 'habrían', 'hubieras', 'habidas', 'tendremos', 'habíais', 'tuviste', 'estuvieseis', 'estada', 'soy', '#', 'seré', 'tuvisteis', 'fuésemos', 'nosotras', 'quienes', 'hubiéramos', 'ese', 'le', 'tendrás', 'habrás', 'esos', 'tuviese', 'en', 'nuestras', 'tenidas', 'estando', 'sentida', 'entre', 'nos', 'son', 'estéis', 'tuvierais', 'durante', 'estaréis', 'nuestra', 'cual', 'suyos', '=', 'esas', 'las', 'mías', 'fuerais', 'desde', 'tendría', '<', 'unos', 'fuesen', 'cuando', 'ti', 'estarías', 'otra', 'fuéramos', 'habiendo', 'teniendo', 'tuvieras', 'tenéis', ',', '{', 'más', 'tuya', 'tú', 'mío', 'habría', ';', 'tened', ':', 'tengan', ')', '~', 'tuyos', 'habido', 'tenía', 'fuese', 'estaban', 'sentido', 'estuvieron', 'porque', 'un', 'tuyo', 'esa', 'estadas', 'hayamos', '

In [410]:
print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names())
print(vector.shape)
print(vector.toarray())

{'debugear': 527, 'código': 504, 'django': 635, 'utilizando': 1958, 'docker': 639, 'file.conf': 820, 'muestra': 1261, 'error': 742, 'usando': 1948, 'supervisor': 1822, 'digital': 608, 'ocean': 1336, '¿alguna': 2080, 'opcion': 1352, 'corregirlo': 462, 'ejecutar': 686, 'navegador': 1291, 'pantalla': 1383, 'completa': 382, 'iniciar': 1010, 's.o': 1666, 'linux': 1140, 'pagina': 1379, 'web': 2029, 'blanco': 248, 'obtengo': 1333, 'i/choreographer': 958, 'skipped': 1754, 'frames': 850, 'the': 1873, 'application': 166, 'may': 1204, 'be': 241, 'doing': 645, 'too': 1891, 'much': 1258, 'work': 2049, 'on': 1350, 'its': 1063, 'main': 1180, 'thread': 1875, 'java': 1067, 'android': 135, 'generar': 879, 'documento': 642, 'partir': 1400, 'plantilla': 1454, '.docx': 42, 'comando': 363, '¿error': 2097, 'realizar': 1572, 'deploy': 557, 'heroku': 936, 'desinstalar': 580, 'lanzar': 1098, 'comandos': 364, 'bloqueo': 254, 'proyectos': 1518, 'netbeans': 1299, 'centos': 319, '¿como': 2083, 'configurar': 414, 'b

In [400]:
sparse_matrix = vectorizer.fit_transform(test)
print(sparse_matrix[0:1])

  (0, 527)	1
  (0, 504)	1
  (0, 635)	1
  (0, 1958)	1
  (0, 639)	1


In [401]:
doc_term_matrix = sparse_matrix.todense()
doc_term_matrix

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:

X = vectorizer.fit_transform(corpus)

In [402]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(sparse_matrix, sparse_matrix))

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.18898224 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.18898224 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [407]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(doc_term_matrix[232], doc_term_matrix).flatten()
print(cosine_similarities)

[0. 1. 0. ... 0. 0. 0.]


In [413]:
related_docs_indices = cosine_similarities.argsort()[:-6:-1]
print(related_docs_indices)

print(cosine_similarities[related_docs_indices])


[232 842  65 490 457]
[7. 3. 2. 2. 2.]


In [414]:
print(test[232])
print(test[842])
print(test[65])
print(test[490])
print(test[457])

Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
Tengo un error constante en Debian cuando quiero actualizar mi sistema
¿Por qué me da el error "syntax error: unexpected end of file"?
¿Ayuda me da este error al hacer en consulta en Mysql Uncaught Error: Call to undefined function mysql_error()?
Error al instalar php-db en Debian 10 Buster


TEST - SOFT COSINE